# Passo 1. Carregar base de dados

In [ ]:
"""### Importacao de bibliotecas"""
import numpy as np
import pandas as pd

In [ ]:
#Carregar a base e visualizar primeiras linhas
data = pd.read_csv('LifeExpectancy.csv', index_col=0)
#Visualize a base pelo Explorador de Variáveis
data.head()

In [ ]:
import missingno as msno
msno.matrix(data, figsize=(15,6));

In [ ]:
#dimnesões da base
data.shape

In [ ]:
data.describe()

In [ ]:
#separa dados a serem inferidos (LifeExpectancy = NaN)
filtro = data['LifeExpectancy'].isnull()
data_infer = data[filtro]
data_infer.shape

In [ ]:
data_infer.describe()

In [ ]:
#retirar dados com missing no rótulo da base a ser usada pra treinar e testar
data = data[~filtro]
data.shape

In [ ]:
data.describe()

In [ ]:
data.Status.unique()

In [ ]:
data.Status.value_counts()

In [ ]:
# OneHotEncoding
from sklearn.preprocessing import OneHotEncoder
onehotencoder = OneHotEncoder(drop='first') #deleta uma das colunas. evita multicolinearidade
onehotencoder = onehotencoder.fit(data[['Status']])
data.loc[:,'Status'] = onehotencoder.transform(data[['Status']]).toarray() 

data.Status.value_counts()

In [ ]:
data.head()
#agora toda base é numérica!

#Passo 2. Separar base em treino e teste

In [ ]:
np.random.seed(0) #semente inicial

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data.loc[:,data.columns != 'LifeExpectancy'], 
                                                    data['LifeExpectancy'], 
                                                    test_size=0.2)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
# Tratar missing
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=2)
imputer = imputer.fit(X_train)
X_train = imputer.transform(X_train)
X_test = imputer.transform(X_test)

In [ ]:
msno.matrix(pd.DataFrame(X_train), figsize=(15,5))

In [ ]:
"""## Análise exploratória"""
import seaborn as sns
corr = data.corr()
sns.heatmap(corr, 
        xticklabels=corr.columns,
        yticklabels=corr.columns,
        vmin=-1, vmax=1);

# Passo 3. Normalizar Dados

In [ ]:
"""## **Normalizando os Dados**"""
from sklearn import preprocessing
preprocessParams = preprocessing.StandardScaler().fit(X_train)
X_train_normalized = preprocessParams.transform(X_train)
X_test_normalized = preprocessParams.transform(X_test)

X_train_normalized[0:2, :]

# Passo 4. Treinar Rede Neural

In [ ]:
"""# Rede Neural"""
from keras import Sequential
from keras.layers import Dense

RN = Sequential()
RN.add(Dense(10, input_shape = X_train_normalized.shape[1:], activation = 'sigmoid'))
RN.add(Dense(1)) #funcao de ativacao eh linear
RN.summary()

In [ ]:
# treinamento
RN.compile(optimizer = 'sgd', loss = 'mean_squared_error', metrics = ['mape']) #se fosse classificacao: categorical_cross_entropy
history = RN.fit(X_train_normalized,y_train, epochs = 100)

# Passo 5. Avaliação da Rede Neural

In [ ]:
# Gráfico do treinamento: custo do treino
import matplotlib.pyplot as plt
plt.plot(history.history['loss'], label='train')
plt.title('Loss de treino')
plt.ylabel('loss')
plt.xlabel('época')
plt.legend();

In [ ]:
# avaliação
score = RN.evaluate(X_test_normalized, y_test, verbose = 0)
print('Test score:', score[0])
print('Test mape:', score[1], '%')

# Passo 6. Fazer novas previsões

In [ ]:
# Fazer as previsões
y_test_predicted = RN.predict(X_test_normalized)

In [ ]:
# Avaliação gráfica
results = pd.DataFrame(y_test_predicted, columns = ['previsto'])
results['real'] = np.array(y_test)
results.plot(figsize=(20,5)); #todas as previsões

In [ ]:
results.iloc[0:100,:].plot(figsize=(20,5));

In [ ]:
from sklearn.metrics import r2_score
r2_score(y_test, y_test_predicted)

#Inferência para data_infer (sem rótulos)

In [ ]:
# retirar coluna LifeExpectancy (não tem nada mesmo...)
data_infer = data_infer.drop(columns=['LifeExpectancy'])

# salvar nomes dos  países
countries = data_infer.index

# 1.onehotencoding
data_infer['Status'] = onehotencoder.transform(data_infer[['Status']]).toarray()

In [ ]:
# 2.tratamento de missing
data_infer = imputer.transform(data_infer)

In [ ]:
# 3.Normalização
data_infer_normalized = preprocessParams.transform(data_infer)

In [ ]:
# 4.Previsão
y_pred = RN.predict(data_infer_normalized)
y_pred

In [ ]:
# 5.Gráfico
plt.scatter(countries, y_pred)
plt.xticks(rotation=90)
plt.title('Previsões')
plt.ylabel('Expectativa de vida');
plt.show()

In [ ]:
# 5.Gráfico
plt.bar(countries, y_pred.ravel())
plt.xticks(rotation=90)
plt.title('Previsões')
plt.ylabel('Expectativa de vida');
plt.show()

1. Separe 20% da base para validação
2. Plote o loss de validação juntamente com o loss de treino
3. Plote o MAPE de treino e validação
4. Faça um gráfico de dispersão onde x=y_test e y=y_test_predicted. Indique a reta x=y (ou seja, os pontos onde a previsão seria perfeita).
5. Aumente o número de épocas.

**solução no script 'ExpectativaDeVida_solucao.ipynb'